<a href="https://colab.research.google.com/github/jualvessan/Simulador_Entrevistas/blob/main/simular_entrevista_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# -*- coding: utf-8 -*-
"""Simulador_Entrevista.ipynb

Automatically generated by Colab.

Original file is located at
    https://colab.research.google.com/drive/1Yjnf-rFZJPM_zVJ4qD11tWaZ0IO8xpba

Simulador de Entrevistas com Feedback Personalizado.

Função: Praticar entrevistas de emprego para diversos cargos, recebendo feedback detalhado sobre a comunicação, postura e conteúdo das respostas.



Passo 1: Instalar o SDK do Google

In [2]:
# Import Model
import google.generativeai as genai
# Imports para uso de técnicas PLN
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.tag import pos_tag
# Import request
import requests
# Used to securely store your API key
from google.colab import userdata
api_key = userdata.get("SECRET_KEY")
genai.configure(api_key=api_key)

Passo 1.1 : Download dos pacotes necessários para o desenvolvimento com técnicas de linguagem natual

In [3]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

Passo 2: Configuração de variável candidata, temperatura e configurações de segurança

In [4]:
generation_config = {
  "candidate_count":1,
  "temperature": 0.5
}

Passo 3: Inicializar o modelo

In [5]:
model = genai.GenerativeModel(model_name = "gemini-1.0-pro",
                              generation_config = generation_config)

Passo 4: Manter o histórico do chat

In [6]:
chat = model.start_chat(history=[])

Passo 5: Definir dicionário de habilidades técnicas por cargo

In [7]:
habilidades_tecnicas = {
    "Desenvolvedor Python": ["Python", "Django", "Flask", "SQL", "Git"],
    "Analista de Dados": ["R", "SQL", "Pandas", "Machine Learning", "Tableau"],
    "Designer UX/UI": ["Figma", "Adobe XD", "Sketch", "UI/UX Principles", "User Research"],
}

"""Passo 6: Definir dicionário de habilidades comportamentais por cargo"""

habilidades_comportamentais = {
    "Desenvolvedor Python": ["Comunicação clara", "Resolução de problemas", "Aprendizagem contínua", "Trabalho em equipe", "Gerenciamento de tempo"],
    "Analista de Dados": ["Orientação para detalhes", "Pensamento analítico", "Habilidades de comunicação", "Criatividade", "Adaptabilidade"],
    "Designer UX/UI": ["Empatia", "Criatividade", "Comunicação visual", "Resolução de problemas", "Trabalho em equipe"],
}

Passo 7: Função para gerar feedback personalizado

In [8]:
def gerar_feedback(resposta, habilidade, vaga):
    # Pré-processamento
    resposta_limpa = resposta.lower().replace(".", "").strip()
    habilidade_limpa = " ".join(habilidade).lower().replace(".", "").strip()
    resposta_tokenizada = word_tokenize(resposta_limpa)
    habilidade_tokenizada = word_tokenize(habilidade_limpa)

    # Análise sintática
    resposta_pos_tags = pos_tag(resposta_tokenizada)
    habilidade_pos_tags = pos_tag(habilidade_tokenizada)

    # Análise semântica
    pontos_chave = {}  # Dicionário para armazenar pontos fortes e fracos
    for palavra_chave, tag in habilidade_pos_tags:
        if palavra_chave in resposta_tokenizada:
            pontos_chave[palavra_chave] = "forte"
        else:
            pontos_chave[palavra_chave] = "fraco"

    # Geração de feedback
    feedback = f"Sua resposta: {resposta}\n\n"

    if pontos_chave:
        pontos_fortes = [chave for chave, valor in pontos_chave.items() if valor == "forte"]
        if pontos_fortes:
            feedback += f" Para a vaga escolhida, você declarou conhecimento em: {', '.join(pontos_fortes)}\n\n"

        pontos_fracos = [chave for chave, valor in pontos_chave.items() if valor == "fraco"]
        if pontos_fracos:
            feedback += f" Contudo, para a vaga de {vaga}, seria interessante demonstrar conhecimento em: {', '.join(pontos_fracos)}\n\n"
            feedback += " Que tal se aprofundar nesses tópicos? Aqui estão algumas sugestões de cursos:\n\n"

            for palavra_chave in pontos_fracos:
                response = model.generate_content(f"Liste 3 cursos online sobre {palavra_chave}")
                if response:
                    feedback += f"{response.text}\n\n"

    # Sugestão de resposta com base nas habilidades da vaga
    feedback += "**Que tal tentar esta resposta?**\n\n"
    response = model.generate_content(f"Crie uma resposta para a pergunta, mencionando as seguintes habilidades: {', '.join(habilidade)}")
    if response:
        feedback += f"{response.text}\n\n"

    feedback += "Lembre-se: Adapte sua resposta de acordo com as necessidades específicas da vaga e da empresa, além da sua própria experiência."

    return feedback

Passo 8: Função para simular entrevista

In [10]:
def simular_entrevista(vaga):
    # Definir perguntas de acordo com a vaga
    if vaga == "Desenvolvedor Python":
        perguntas = [
            "Me fale sobre sua experiência como Desenvolvedor Python.",
            "Qual projeto Python do qual você mais se orgulha?",
            "Como você resolveria um erro de codificação inesperado?",
            "Como você se mantém atualizado com as últimas tecnologias em desenvolvimento?",
            "Conte-me sobre um desafio que você superou em um projeto de desenvolvimento.",
        ]
        habilidade = habilidades_tecnicas[vaga]
    elif vaga == "Analista de Dados":
        perguntas = [
            "Descreva sua experiência com análise de dados.",
            "Qual ferramenta de análise de dados você mais utiliza?",
            "Como você comunicaria os resultados de uma análise de dados para stakeholders não técnicos?",
            "Descreva um projeto de análise de dados que você realizou e qual foi o impacto do projeto.",
            "Como você se manteria atualizado com as últimas tendências em análise de dados?",
        ]
        habilidade = habilidades_tecnicas[vaga]
    elif vaga == "Designer UX/UI":
        perguntas = [
            "Descreva sua experiência com design UX/UI.",
            "Qual software de design UX/UI você mais utiliza?",
            "Como você realiza a pesquisa do usuário para seus projetos?",
            "Descreva um projeto de design UX/UI que você realizou e quais foram os principais desafios.",
            "Como você se manteria atualizado com as últimas tendências em design UX/UI?",
        ]
        habilidade = habilidades_tecnicas[vaga]
    else:
        perguntas = ["Descreva sua experiência profissional.",
                     "Quais são seus pontos fortes e fracos como profissional?",
                     "Por que você se interessa por essa vaga?",
        ]
        habilidade = ["Experiência profissional", "Pontos fortes", "Pontos fracos", "Interesse na vaga"] # Habilidades genéricas para outros cargos

    for pergunta in perguntas:
        print("Recrutador(a): " + pergunta + "\n> ") # Perguntas cadastradas
        resposta = input("Digite sua resposta: " + "\n> ")  # Receber resposta do candidato

        feedback = gerar_feedback(resposta, habilidade, vaga)
        print(feedback + "\n> ")  # Exibir feedback para o candidato

    return simular_entrevista

Passo 9: Chamada das funções. Execução do programa

In [11]:
# Chamada da função simular_entrevista

print("Iniciando o simulador de entrevista...." + "\n> ")
vaga_escolhida = prompt = input("Primeiro informe uma vaga que deseja se candidatar:  " + "\n> ")
print("Vaga Escolhida: " + vaga_escolhida + "\n> ")

simular_entrevista = simular_entrevista(vaga_escolhida)

Iniciando o simulador de entrevista....
> 
Primeiro informe uma vaga que deseja se candidatar:  
> Desenvolvedor Python
Vaga Escolhida: Desenvolvedor Python
> 
Recrutador(a): Me fale sobre sua experiência como Desenvolvedor Python.
> 
Digite sua resposta: 
> Sou programadora junior em python
Sua resposta: Sou programadora junior em python

 Para a vaga escolhida, você declarou conhecimento em: python

 Contudo, para a vaga de Desenvolvedor Python, seria interessante demonstrar conhecimento em: django, flask, sql, git

 Que tal se aprofundar nesses tópicos? Aqui estão algumas sugestões de cursos:

1. **Django para Principiantes** (Udemy)
2. **Tutorial Completo de Django 4** (Coursera)
3. **Django: El Marco Web de Python para Principiantes** (edX)

1. **Flask Mega-Tutorial** (Udemy)
2. **Flask Web Development: Building Websites with Python** (Coursera)
3. **Flask Web Development with Python** (edX)

1. **SQL para Principiantes** (Coursera)
2. **Fundamentos de SQL** (edX)
3. **SQL: El Len